# TCN

In [ ]:
modelname = "TCN"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\TCN_07.03_Test_Results\TCN_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["TCN_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹
    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e5                # € per unserved kWh

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        
        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_TCN"]   = Pb
        blk["SOC"]               = SOCd
        blk["Grid_Load_kW_TCN"] = Gr
        chunks.append(blk)

    forecast_df = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 POST-EVALUATION WITH ACTUAL LOAD
    # ----------------------------------------------------------------
    forecast_df["Grid_Load_kW_actual"] = np.maximum(
            forecast_df["Actual Load"] - forecast_df["Battery_kW_TCN"], 0)

    Grid_eval = forecast_df["Grid_Load_kW_actual"].values
    price     = forecast_df["Price"].values
    load_act  = forecast_df["Actual Load"].values

    cost_no_batt   = Delta * np.sum(load_act  * price)
    cost_with_batt = Delta * np.sum(Grid_eval * price)

    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})


In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\TCN"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))

# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:blue", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (TCN)")
plt.tight_layout()
plt.show()


In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\TCN_07.03_Test_Results\TCN_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["TCN_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
Delta = 0.25   # h, kWh, h⁻¹
Etot = 12  # h, kWh, h⁻¹
C_rate = 0.25   # h, kWh, h⁻¹
Pmax               = Etot * C_rate       # 5 kW
initial_SOC        = 0.10
final_soc_min      = 0.10

demand_charge_rate = 0.0
peak_threshold     = 50.0
peak_penalty_rate  = 15.0
slack_penalty_rate = 1000.0
very_big           = 1e5                # € per unserved kWh

# ----------------------------------------------------------------
# 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
# ----------------------------------------------------------------
def optimise_one_day(load, price, soc_init, p):
    T = len(load)

    # ── decision variables ───────────────────────────────────────
    P_bat  = cp.Variable(T)             # + discharge / – charge
    SOC    = cp.Variable(T)             # state of charge (fraction)
    Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
    Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
    Peak   = cp.Variable()              # daily peak (kW)
    Excess = cp.Variable(T,  nonneg=True)
    slack  = cp.Variable(T,  nonneg=True)

    # ── build constraints one-by-one ─────────────────────────────
    constr = []

    # 1. Battery power limits
    constr += [P_bat >= -p["Pmax"]]     # charging limit
    constr += [P_bat <=  p["Pmax"]]     # discharging limit

    # 2. Initial SOC
    constr += [SOC[0] == soc_init]

    # 3. SOC update for each time step
    for t in range(1, T):
        constr += [
            SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
        ]

    # 4. SOC bounds
    constr += [SOC >= 0]
    constr += [SOC <= 1]

    # 5. Define unmet load  (forecast load – battery discharge)
    constr += [Unsv == load - P_bat]

    # 6. Power balance with **no export**
    constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
    constr += [Grid >= 0]      # import only

    # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
    constr += [P_bat >= -load]        # because load ≥ 0
    
    # 7. Optional peak-cap with slack
    peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
    constr += [Grid <= peak_cap + slack]

    # 8. Peak variable must be ≥ every grid value
    constr += [Peak >= Grid]

    # 9. Excess above hard threshold (for penalties)
    constr += [Excess >= Grid - p["peak_threshold"]]
    constr += [Excess >= 0]

    # 10. End-of-day minimum SOC
    constr += [SOC[-1] >= p["soc_final_min"]]

    # ── objective function ───────────────────────────────────────
    energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
    demand_charge = p["demand_rate"] * Peak
    soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
    slack_pen     = p["slack_cost"] * cp.sum(slack)
    unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

    objective = cp.Minimize(
        energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
    )

    # ── solve ────────────────────────────────────────────────────
    cp.Problem(objective, constr).solve(solver=cp.ECOS, verbose=False)

    return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


# ----------------------------------------------------------------
# 5 ROLLING-HORIZON LOOP
# ----------------------------------------------------------------
window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
chunks, soc0 = [], initial_SOC

for d0 in window[:-1]:
    d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
    mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

    Pb, SOCd, Gr, soc0 = optimise_one_day(
        forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
        forecast_df.loc[mask, "Price"].values,
        soc0,
        dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
             demand_rate=demand_charge_rate,
             peak_threshold=peak_threshold,
             penalty_rate=peak_penalty_rate,
             soc_final_min=final_soc_min,
             orig_peak=forecast_df["Actual Load"].max(),
             slack_cost=slack_penalty_rate)
    )

    blk = forecast_df.loc[mask].copy()
    blk["Battery_kW_TCN"]   = Pb
    blk["SOC"]               = SOCd
    blk["Grid_Load_kW_TCN"] = Gr
    chunks.append(blk)

forecast_df = pd.concat(chunks, ignore_index=True)

# ----------------------------------------------------------------
# 6 POST-EVALUATION WITH ACTUAL LOAD
# ----------------------------------------------------------------
forecast_df["Grid_Load_kW_actual"] = np.maximum(
        forecast_df["Actual Load"] - forecast_df["Battery_kW_TCN"], 0)

Grid_eval = forecast_df["Grid_Load_kW_actual"].values
price     = forecast_df["Price"].values
load_act  = forecast_df["Actual Load"].values

cost_no_batt   = Delta * np.sum(load_act  * price)
cost_with_batt = Delta * np.sum(Grid_eval * price)

print("────────────────────────────────────────────────")
print(f"📈 Profit (bill reduction): €{cost_no_batt-cost_with_batt:,.2f} "
      f"({100*(cost_no_batt-cost_with_batt)/cost_no_batt:.2f} %)")
print("────────────────────────────────────────────────\n")

# ----------------------------------------------------------------
# 7  PLOTS   (now 5 rows)
# ----------------------------------------------------------------
plt.figure(figsize=(14, 15))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(5,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Forecast_kW"],
         label="Forecast (TCN)", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(5,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Battery_kW_TCN"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kWh"); plt.grid(True)

# 3 ▶ State-of-Charge
plt.subplot(5,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid(True)

# 4 ▶ NEW: Actual load time-series
plt.subplot(5,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Actual Load"],
         label="Actual Load", color="grey")
plt.title("Actual Building Load"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 5 ▶ Grid import after battery operation
plt.subplot(5,1,5)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Grid_Load_kW_TCN"],
         label="Grid Import (actual)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

plt.tight_layout()
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\Dispatch_plots"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "TCN.png")
plt.savefig(save_path, dpi=600)
plt.show()

# LSTM

In [ ]:
modelname = "LSTM"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\LSTM_14.03_Test_Results\LSTM_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["LSTM_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹
    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e5                # € per unserved kWh

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        


        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_LSTM"]   = Pb
        blk["SOC"]               = SOCd
        blk["Grid_Load_kW_LSTM"] = Gr
        chunks.append(blk)

    forecast_df = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 POST-EVALUATION WITH ACTUAL LOAD
    # ----------------------------------------------------------------
    forecast_df["Grid_Load_kW_actual"] = np.maximum(
            forecast_df["Actual Load"] - forecast_df["Battery_kW_LSTM"], 0)

    Grid_eval = forecast_df["Grid_Load_kW_actual"].values
    price     = forecast_df["Price"].values
    load_act  = forecast_df["Actual Load"].values

    cost_no_batt   = Delta * np.sum(load_act  * price)
    cost_with_batt = Delta * np.sum(Grid_eval * price)

    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})


In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\LSTM"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))

# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:blue", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (LSTM)")
plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\TSMixerx_11.03_Test_Results\TSMixerx_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["TSMixerx_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
Delta = 0.25   # h, kWh, h⁻¹
Etot = 12  # h, kWh, h⁻¹
C_rate = 0.25   # h, kWh, h⁻¹
Pmax               = Etot * C_rate       # 5 kW
initial_SOC        = 0.10
final_soc_min      = 0.10

demand_charge_rate = 0.0
peak_threshold     = 50.0
peak_penalty_rate  = 15.0
slack_penalty_rate = 1000.0
very_big           = 1e5                # € per unserved kWh

# ----------------------------------------------------------------
# 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
# ----------------------------------------------------------------
def optimise_one_day(load, price, soc_init, p):
    T = len(load)

    # ── decision variables ───────────────────────────────────────
    P_bat  = cp.Variable(T)             # + discharge / – charge
    SOC    = cp.Variable(T)             # state of charge (fraction)
    Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
    Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
    Peak   = cp.Variable()              # daily peak (kW)
    Excess = cp.Variable(T,  nonneg=True)
    slack  = cp.Variable(T,  nonneg=True)

    # ── build constraints one-by-one ─────────────────────────────
    constr = []

    # 1. Battery power limits
    constr += [P_bat >= -p["Pmax"]]     # charging limit
    constr += [P_bat <=  p["Pmax"]]     # discharging limit

    # 2. Initial SOC
    constr += [SOC[0] == soc_init]

    # 3. SOC update for each time step
    for t in range(1, T):
        constr += [
            SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
        ]

    # 4. SOC bounds
    constr += [SOC >= 0]
    constr += [SOC <= 1]

    # 5. Define unmet load  (forecast load – battery discharge)
    constr += [Unsv == load - P_bat]

    # 6. Power balance with **no export**
    constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
    constr += [Grid >= 0]      # import only

    # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
    constr += [P_bat >= -load]        # because load ≥ 0
    
    # 7. Optional peak-cap with slack
    peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
    constr += [Grid <= peak_cap + slack]

    # 8. Peak variable must be ≥ every grid value
    constr += [Peak >= Grid]

    # 9. Excess above hard threshold (for penalties)
    constr += [Excess >= Grid - p["peak_threshold"]]
    constr += [Excess >= 0]

    # 10. End-of-day minimum SOC
    constr += [SOC[-1] >= p["soc_final_min"]]

    # ── objective function ───────────────────────────────────────
    energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
    demand_charge = p["demand_rate"] * Peak
    soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
    slack_pen     = p["slack_cost"] * cp.sum(slack)
    unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

    objective = cp.Minimize(
        energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
    )

    # ── solve ────────────────────────────────────────────────────
    cp.Problem(objective, constr).solve(solver=cp.ECOS, verbose=False)

    return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


# ----------------------------------------------------------------
# 5 ROLLING-HORIZON LOOP
# ----------------------------------------------------------------
window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
chunks, soc0 = [], initial_SOC

for d0 in window[:-1]:
    d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
    mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

    Pb, SOCd, Gr, soc0 = optimise_one_day(
        forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
        forecast_df.loc[mask, "Price"].values,
        soc0,
        dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
             demand_rate=demand_charge_rate,
             peak_threshold=peak_threshold,
             penalty_rate=peak_penalty_rate,
             soc_final_min=final_soc_min,
             orig_peak=forecast_df["Actual Load"].max(),
             slack_cost=slack_penalty_rate)
    )

    blk = forecast_df.loc[mask].copy()
    blk["Battery_kW_LSTM"]   = Pb
    blk["SOC"]               = SOCd
    blk["Grid_Load_kW_LSTM"] = Gr
    chunks.append(blk)

forecast_df = pd.concat(chunks, ignore_index=True)

# ----------------------------------------------------------------
# 6 POST-EVALUATION WITH ACTUAL LOAD
# ----------------------------------------------------------------
forecast_df["Grid_Load_kW_actual"] = np.maximum(
        forecast_df["Actual Load"] - forecast_df["Battery_kW_LSTM"], 0)

Grid_eval = forecast_df["Grid_Load_kW_actual"].values
price     = forecast_df["Price"].values
load_act  = forecast_df["Actual Load"].values

cost_no_batt   = Delta * np.sum(load_act  * price)
cost_with_batt = Delta * np.sum(Grid_eval * price)

print("────────────────────────────────────────────────")
print(f"📈 Profit (bill reduction): €{cost_no_batt-cost_with_batt:,.2f} "
      f"({100*(cost_no_batt-cost_with_batt)/cost_no_batt:.2f} %)")
print("────────────────────────────────────────────────\n")

# ----------------------------------------------------------------
# 7  PLOTS   (now 5 rows)
# ----------------------------------------------------------------
plt.figure(figsize=(14, 15))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(5,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Forecast_kW"],
         label="Forecast (LSTM)", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(5,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Battery_kW_LSTM"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kWh"); plt.grid(True)

# 3 ▶ State-of-Charge
plt.subplot(5,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid(True)

# 4 ▶ NEW: Actual load time-series
plt.subplot(5,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Actual Load"],
         label="Actual Load", color="grey")
plt.title("Actual Building Load"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 5 ▶ Grid import after battery operation
plt.subplot(5,1,5)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Grid_Load_kW_LSTM"],
         label="Grid Import (actual)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

plt.tight_layout()
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\Dispatch_plots"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "LSTM.png")
plt.savefig(save_path, dpi=600)
plt.show()

# KAN

In [ ]:
modelname = "KAN"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\KAN_14.03_Test_Results\KAN_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["KAN_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹
    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e5                # € per unserved kWh

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        

        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_KAN"]   = Pb
        blk["SOC"]               = SOCd
        blk["Grid_Load_kW_KAN"] = Gr
        chunks.append(blk)

    forecast_df = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 POST-EVALUATION WITH ACTUAL LOAD
    # ----------------------------------------------------------------
    forecast_df["Grid_Load_kW_actual"] = np.maximum(
            forecast_df["Actual Load"] - forecast_df["Battery_kW_KAN"], 0)

    Grid_eval = forecast_df["Grid_Load_kW_actual"].values
    price     = forecast_df["Price"].values
    load_act  = forecast_df["Actual Load"].values

    cost_no_batt   = Delta * np.sum(load_act  * price)
    cost_with_batt = Delta * np.sum(Grid_eval * price)

    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})


In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\KAN"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")
for spine in ['top', 'right', 'left']:
    ax1.spines[spine].set_visible(False)

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:orange", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:orange")
ax2.tick_params(axis="y", labelcolor="tab:orange")
for spine in ['top', 'right', 'left']:
    ax2.spines[spine].set_visible(False)

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (KAN)")
plt.tight_layout()
plt.grid(True)
plt.show()



In [ ]:
plt.figure(figsize=(14, 14))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(5,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Forecast_kW"],
         label="Forecast (KAN)", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kW"); plt.grid(); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(5,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Battery_kW_KAN"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kW"); plt.grid()

# 3 ▶ State-of-Charge
plt.subplot(5,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid()

# 4 ▶ NEW: Actual load time-series
plt.subplot(5,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Actual Load"],
         label="Actual Load", color="grey")
plt.title("Actual Building Load"); plt.ylabel("kW"); plt.grid(); plt.legend()

# 5 ▶ Grid import after battery operation
plt.subplot(5,1,5)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Grid_Load_kW_actual"],
         label="Grid Import (actual)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kW"); plt.grid(); plt.legend()

plt.tight_layout()
plt.show()

# TSMixersx

In [ ]:
modelname = "TSMixersx"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\TSMixerx_11.03_Test_Results\TSMixerx_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["TSMixerx_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹
    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e5                # € per unserved kWh

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        


        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_TSMixerx"]   = Pb
        blk["SOC"]               = SOCd
        blk["Grid_Load_kW_TSMixerx"] = Gr
        chunks.append(blk)

    forecast_df = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 POST-EVALUATION WITH ACTUAL LOAD
    # ----------------------------------------------------------------
    forecast_df["Grid_Load_kW_actual"] = np.maximum(
            forecast_df["Actual Load"] - forecast_df["Battery_kW_TSMixerx"], 0)

    Grid_eval = forecast_df["Grid_Load_kW_actual"].values
    price     = forecast_df["Price"].values
    load_act  = forecast_df["Actual Load"].values

    cost_no_batt   = Delta * np.sum(load_act  * price)
    cost_with_batt = Delta * np.sum(Grid_eval * price)

    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})


In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\TSMixersx"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))

# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:blue", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (TSMixersx)")
plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\TSMixerx_11.03_Test_Results\TSMixerx_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["TSMixerx_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
Delta = 0.25   # h, kWh, h⁻¹
Etot = 12  # h, kWh, h⁻¹
C_rate = 0.25   # h, kWh, h⁻¹
Pmax               = Etot * C_rate       # 5 kW
initial_SOC        = 0.10
final_soc_min      = 0.10

demand_charge_rate = 0.0
peak_threshold     = 50.0
peak_penalty_rate  = 15.0
slack_penalty_rate = 1000.0
very_big           = 1e5                # € per unserved kWh

# ----------------------------------------------------------------
# 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
# ----------------------------------------------------------------
def optimise_one_day(load, price, soc_init, p):
    T = len(load)

    # ── decision variables ───────────────────────────────────────
    P_bat  = cp.Variable(T)             # + discharge / – charge
    SOC    = cp.Variable(T)             # state of charge (fraction)
    Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
    Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
    Peak   = cp.Variable()              # daily peak (kW)
    Excess = cp.Variable(T,  nonneg=True)
    slack  = cp.Variable(T,  nonneg=True)

    # ── build constraints one-by-one ─────────────────────────────
    constr = []

    # 1. Battery power limits
    constr += [P_bat >= -p["Pmax"]]     # charging limit
    constr += [P_bat <=  p["Pmax"]]     # discharging limit

    # 2. Initial SOC
    constr += [SOC[0] == soc_init]

    # 3. SOC update for each time step
    for t in range(1, T):
        constr += [
            SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
        ]

    # 4. SOC bounds
    constr += [SOC >= 0]
    constr += [SOC <= 1]

    # 5. Define unmet load  (forecast load – battery discharge)
    constr += [Unsv == load - P_bat]

    # 6. Power balance with **no export**
    constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
    constr += [Grid >= 0]      # import only

    # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
    constr += [P_bat >= -load]        # because load ≥ 0
    
    # 7. Optional peak-cap with slack
    peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
    constr += [Grid <= peak_cap + slack]

    # 8. Peak variable must be ≥ every grid value
    constr += [Peak >= Grid]

    # 9. Excess above hard threshold (for penalties)
    constr += [Excess >= Grid - p["peak_threshold"]]
    constr += [Excess >= 0]

    # 10. End-of-day minimum SOC
    constr += [SOC[-1] >= p["soc_final_min"]]

    # ── objective function ───────────────────────────────────────
    energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
    demand_charge = p["demand_rate"] * Peak
    soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
    slack_pen     = p["slack_cost"] * cp.sum(slack)
    unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

    objective = cp.Minimize(
        energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
    )

    # ── solve ────────────────────────────────────────────────────
    cp.Problem(objective, constr).solve(solver=cp.ECOS, verbose=False)

    return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


# ----------------------------------------------------------------
# 5 ROLLING-HORIZON LOOP
# ----------------------------------------------------------------
window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
chunks, soc0 = [], initial_SOC

for d0 in window[:-1]:
    d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
    mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

    Pb, SOCd, Gr, soc0 = optimise_one_day(
        forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
        forecast_df.loc[mask, "Price"].values,
        soc0,
        dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
             demand_rate=demand_charge_rate,
             peak_threshold=peak_threshold,
             penalty_rate=peak_penalty_rate,
             soc_final_min=final_soc_min,
             orig_peak=forecast_df["Actual Load"].max(),
             slack_cost=slack_penalty_rate)
    )

    blk = forecast_df.loc[mask].copy()
    blk["Battery_kW_TSMixerx"]   = Pb
    blk["SOC"]               = SOCd
    blk["Grid_Load_kW_TSMixerx"] = Gr
    chunks.append(blk)

forecast_df = pd.concat(chunks, ignore_index=True)

# ----------------------------------------------------------------
# 6 POST-EVALUATION WITH ACTUAL LOAD
# ----------------------------------------------------------------
forecast_df["Grid_Load_kW_actual"] = np.maximum(
        forecast_df["Actual Load"] - forecast_df["Battery_kW_TSMixerx"], 0)

Grid_eval = forecast_df["Grid_Load_kW_actual"].values
price     = forecast_df["Price"].values
load_act  = forecast_df["Actual Load"].values

cost_no_batt   = Delta * np.sum(load_act  * price)
cost_with_batt = Delta * np.sum(Grid_eval * price)

print("────────────────────────────────────────────────")
print(f"📈 Profit (bill reduction): €{cost_no_batt-cost_with_batt:,.2f} "
      f"({100*(cost_no_batt-cost_with_batt)/cost_no_batt:.2f} %)")
print("────────────────────────────────────────────────\n")

# ----------------------------------------------------------------
# 7  PLOTS   (now 5 rows)
# ----------------------------------------------------------------
plt.figure(figsize=(14, 15))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(5,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Forecast_kW"],
         label="Forecast (TSMixerx)", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(5,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Battery_kW_TSMixerx"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kWh"); plt.grid(True)

# 3 ▶ State-of-Charge
plt.subplot(5,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid(True)

# 4 ▶ NEW: Actual load time-series
plt.subplot(5,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Actual Load"],
         label="Actual Load", color="grey")
plt.title("Actual Building Load"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 5 ▶ Grid import after battery operation
plt.subplot(5,1,5)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Grid_Load_kW_TSMixerx"],
         label="Grid Import (actual)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

plt.tight_layout()
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\Dispatch_plots"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "TSMixerx.png")
plt.savefig(save_path, dpi=600)
plt.show()


In [ ]:
modelname = 'TSMixersx'
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\TSMixersx"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_optimized_12kW.csv")
forecast_df.to_csv(results_path)
print(f"optimized_12kW saved to {results_path}")


# NBEATSx

In [ ]:
modelname = "NBEATSx"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\NBEATSx_19.05_Test_Results\NBEATSx_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["NBEATSx_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹
    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e5                # € per unserved kWh

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        


        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_NBEATSx"]   = Pb
        blk["SOC"]               = SOCd
        blk["Grid_Load_kW_NBEATSx"] = Gr
        chunks.append(blk)

    forecast_df = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 POST-EVALUATION WITH ACTUAL LOAD
    # ----------------------------------------------------------------
    forecast_df["Grid_Load_kW_actual"] = np.maximum(
            forecast_df["Actual Load"] - forecast_df["Battery_kW_NBEATSx"], 0)

    Grid_eval = forecast_df["Grid_Load_kW_actual"].values
    price     = forecast_df["Price"].values
    load_act  = forecast_df["Actual Load"].values

    cost_no_batt   = Delta * np.sum(load_act  * price)
    cost_with_batt = Delta * np.sum(Grid_eval * price)

    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})


In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\NBEATSx"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))

# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:blue", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (NBEATSx)")
plt.tight_layout()
plt.show()


In [ ]:
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH A – controller sees forecast, result evaluated on actual load
# “cover-as-much-as-possible” + no-export constraint
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\NBEATSx_19.05_Test_Results\NBEATSx_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["NBEATSx_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20


forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
Delta = 0.25   # h, kWh, h⁻¹
Etot = 12  # h, kWh, h⁻¹
C_rate = 0.25   # h, kWh, h⁻¹
Pmax               = Etot * C_rate       # 5 kW
initial_SOC        = 0.10
final_soc_min      = 0.10

demand_charge_rate = 0.0
peak_threshold     = 50.0
peak_penalty_rate  = 15.0
very_big           = 1e5                # € per unserved kWh

# ----------------------------------------------------------------
# 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
# ----------------------------------------------------------------
def optimise_one_day(load, price, soc_init, p):
    T = len(load)

    # ── decision variables ───────────────────────────────────────
    P_bat  = cp.Variable(T)             # + discharge / – charge
    SOC    = cp.Variable(T)             # state of charge (fraction)
    Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
    Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
    Peak   = cp.Variable()              # daily peak (kW)
    Excess = cp.Variable(T,  nonneg=True)
    slack  = cp.Variable(T,  nonneg=True)

    # ── build constraints one-by-one ─────────────────────────────
    constr = []

    # 1. Battery power limits
    constr += [P_bat >= -p["Pmax"]]     # charging limit
    constr += [P_bat <=  p["Pmax"]]     # discharging limit

    # 2. Initial SOC
    constr += [SOC[0] == soc_init]

    # 3. SOC update for each time step
    for t in range(1, T):
        constr += [
            SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
        ]

    # 4. SOC bounds
    constr += [SOC >= 0]
    constr += [SOC <= 1]

    # 5. Define unmet load  (forecast load – battery discharge)
    constr += [Unsv == load - P_bat]

    # 6. Power balance with **no export**
    constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
    constr += [Grid >= 0]      # import only

    # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
    constr += [P_bat >= -load]        # because load ≥ 0
    
    
    # 10. End-of-day minimum SOC
    constr += [SOC[-1] >= p["soc_final_min"]]

    # ── objective function ───────────────────────────────────────
    energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
    demand_charge = p["demand_rate"] * Peak
    soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
    unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

    objective = cp.Minimize(
        energy_cost  + soft_cap_pen  + unserved_pen
    )

    # ── solve ────────────────────────────────────────────────────
    cp.Problem(objective, constr).solve(solver=cp.ECOS, verbose=False)

    return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


# ----------------------------------------------------------------
# 5 ROLLING-HORIZON LOOP
# ----------------------------------------------------------------
window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
chunks, soc0 = [], initial_SOC

for d0 in window[:-1]:
    d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
    mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

    Pb, SOCd, Gr, soc0 = optimise_one_day(
        forecast_df.loc[mask, "Forecast_kW"].values,      # forecast demand
        forecast_df.loc[mask, "Price"].values,
        soc0,
        dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
             demand_rate=demand_charge_rate,
             peak_threshold=peak_threshold,
             penalty_rate=peak_penalty_rate,
             soc_final_min=final_soc_min,
             orig_peak=forecast_df["Actual Load"].max(),
             )
    )

    blk = forecast_df.loc[mask].copy()
    blk["Battery_kW_NBEATSx"]   = Pb
    blk["SOC"]               = SOCd
    blk["Grid_Load_kW_NBEATSx"] = Gr
    chunks.append(blk)

forecast_df = pd.concat(chunks, ignore_index=True)

# ----------------------------------------------------------------
# 6 POST-EVALUATION WITH ACTUAL LOAD
# ----------------------------------------------------------------
forecast_df["Grid_Load_kW_actual"] = np.maximum(
        forecast_df["Actual Load"] - forecast_df["Battery_kW_NBEATSx"], 0)

Grid_eval = forecast_df["Grid_Load_kW_actual"].values
price     = forecast_df["Price"].values
load_act  = forecast_df["Actual Load"].values

cost_no_batt   = Delta * np.sum(load_act  * price)
cost_with_batt = Delta * np.sum(Grid_eval * price)

print("────────────────────────────────────────────────")
print(f"📈 Profit (bill reduction): €{cost_no_batt-cost_with_batt:,.2f} "
      f"({100*(cost_no_batt-cost_with_batt)/cost_no_batt:.2f} %)")
print("────────────────────────────────────────────────\n")

# ----------------------------------------------------------------
# 7  PLOTS   (now 5 rows)
# ----------------------------------------------------------------
plt.figure(figsize=(14, 15))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(5,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Forecast_kW"],
         label="Forecast (NBEATSx)", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(5,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Battery_kW_NBEATSx"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kWh"); plt.grid(True)

# 3 ▶ State-of-Charge
plt.subplot(5,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid(True)

# 4 ▶ NEW: Actual load time-series
plt.subplot(5,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Actual Load"],
         label="Actual Load", color="grey")
plt.title("Actual Building Load"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

# 5 ▶ Grid import after battery operation
plt.subplot(5,1,5)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(forecast_df["Timestamp"], forecast_df["Grid_Load_kW_NBEATSx"],
         label="Grid Import (actual)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kWh"); plt.grid(True); plt.legend()

plt.tight_layout()
plt.show()


# Perfect

In [ ]:
modelname = "Perfect"
import os
import json
import sys

In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH B – optimiser has perfect knowledge of actual load
# Cover-as-much-as-possible, no export
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\LSTM_14.03_Test_Results\LSTM_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["LSTM_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20

forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------
list_of_capacity = range(1,51,1)
results  = []

for i in list_of_capacity:
    Delta = 0.25   # h, kWh, h⁻¹
    Etot = i  # h, kWh, h⁻¹
    C_rate = 0.25   # h, kWh, h⁻¹


    Pmax               = Etot * C_rate       # 5 kW
    initial_SOC        = 0.10
    final_soc_min      = 0.10

    demand_charge_rate = 0.0
    peak_threshold     = 50.0
    peak_penalty_rate  = 15.0
    slack_penalty_rate = 1000.0
    very_big           = 1e4                # €/kWh penalty for unmet load

    # ----------------------------------------------------------------
    # 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
    # ----------------------------------------------------------------
    def optimise_one_day(load, price, soc_init, p):
        T = len(load)

        # ── decision variables ───────────────────────────────────────
        P_bat  = cp.Variable(T)             # + discharge / – charge
        SOC    = cp.Variable(T)             # state of charge (fraction)
        Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
        Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
        Peak   = cp.Variable()              # daily peak (kW)
        Excess = cp.Variable(T,  nonneg=True)
        slack  = cp.Variable(T,  nonneg=True)

        # ── build constraints one-by-one ─────────────────────────────
        constr = []

        # 1. Battery power limits
        constr += [P_bat >= -p["Pmax"]]     # charging limit
        constr += [P_bat <=  p["Pmax"]]     # discharging limit

        # 2. Initial SOC
        constr += [SOC[0] == soc_init]

        # 3. SOC update for each time step
        for t in range(1, T):
            constr += [
                SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
            ]

        # 4. SOC bounds
        constr += [SOC >= 0]
        constr += [SOC <= 1]

        # 5. Define unmet load  (forecast load – battery discharge)
        constr += [Unsv == load - P_bat]

        # 6. Power balance with **no export**
        constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
        constr += [Grid >= 0]      # import only

        # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
        constr += [P_bat >= -load]        # because load ≥ 0
        
        # 7. Optional peak-cap with slack
        peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
        constr += [Grid <= peak_cap + slack]

        # 8. Peak variable must be ≥ every grid value
        constr += [Peak >= Grid]

        # 9. Excess above hard threshold (for penalties)
        constr += [Excess >= Grid - p["peak_threshold"]]
        constr += [Excess >= 0]

        # 10. End-of-day minimum SOC
        constr += [SOC[-1] >= p["soc_final_min"]]

        # ── objective function ───────────────────────────────────────
        energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
        demand_charge = p["demand_rate"] * Peak
        soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
        slack_pen     = p["slack_cost"] * cp.sum(slack)
        unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

        objective = cp.Minimize(
            energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
        )

        # ── solve ────────────────────────────────────────────────────
        prob = cp.Problem(objective, constr)
        prob.solve(solver=cp.ECOS, verbose=False)

        
        return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


    # ----------------------------------------------------------------
    # 5 ROLLING-HORIZON LOOP
    # ----------------------------------------------------------------
    window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
    chunks, soc0 = [], initial_SOC

    for d0 in window[:-1]:
        d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
        mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

        Pb, SOCd, Gr, soc0 = optimise_one_day(
            forecast_df.loc[mask, "Actual Load"].values,      # perfect foresight
            forecast_df.loc[mask, "Price"].values,
            soc0,
            dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
                demand_rate=demand_charge_rate,
                peak_threshold=peak_threshold,
                penalty_rate=peak_penalty_rate,
                soc_final_min=final_soc_min,
                orig_peak=forecast_df["Actual Load"].max(),
                slack_cost=slack_penalty_rate)
        )

        blk = forecast_df.loc[mask].copy()
        blk["Battery_kW_perfect"]   = Pb
        blk["SOC"]                  = SOCd
        blk["Grid_Load_kW_perfect"] = Gr
        chunks.append(blk)

    df_perf = pd.concat(chunks, ignore_index=True)

    # ----------------------------------------------------------------
    # 6 COST CALCULATION  (grid already matches actual load slice)
    # ----------------------------------------------------------------
    price  = df_perf["Price"].values
    load   = df_perf["Actual Load"].values
    grid   = df_perf["Grid_Load_kW_perfect"].values

    cost_no_batt   = Delta * np.sum(load * price)
    cost_with_batt = Delta * np.sum(grid * price)
    save_eur       = cost_no_batt-cost_with_batt
    save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
    print("────────────────────────────────────────────────")
    print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
    print("────────────────────────────────────────────────")
    results.append({"Etot_kWh": Etot,
                    "Saving_€": save_eur,
                    "Saving_%": save_pct})



In [ ]:
df=pd.DataFrame(results)
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\Perfect"
os.makedirs(output_dir, exist_ok=True)
results_path = os.path.join(output_dir, f"{modelname}_Capacity_vs_saving.csv")
df.to_csv(results_path)
print(f"Capacity_vs_saving saved to {results_path}")


In [ ]:
# ------------------------------------------------------------
# 1) Imports
# ------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
from kneed import KneeLocator          # pip install kneed  (optional)

# df = ...   # <- your DataFrame is assumed to be in memory

# ------------------------------------------------------------
# 2) Pick the “optimal” capacities
# ------------------------------------------------------------
# (a) 95 %-of-maximum rule
max_save     = df["Saving_€"].max()
opt95_row    = df.loc[df["Saving_€"] >= 0.95 * max_save].iloc[0]

# (b) Marginal-benefit rule  (threshold = €0.05 per extra kWh)
threshold    = 0.05                      # change if you wish
marginal     = df["Saving_€"].diff() / df["Etot_kWh"].diff()
if (marginal < threshold).any():
    opt_margin_row = df.loc[(marginal < threshold).idxmax()]
else:                                    # never drops below threshold
    opt_margin_row = df.iloc[-1]

# (c) Knee (concave curve, increasing)
try:
    knee      = KneeLocator(df["Etot_kWh"], df["Saving_€"],
                            curve="concave", direction="increasing")
    knee_cap  = knee.knee
    opt_knee_row = df[df["Etot_kWh"] == knee_cap].iloc[0] if knee_cap else None
except Exception:
    knee_cap = None
    opt_knee_row = None

# ------------------------------------------------------------
# 3) Plot
# ------------------------------------------------------------
fig, ax1 = plt.subplots(figsize=(15, 8))

# --- € savings (left y-axis) ---
ax1.plot(df["Etot_kWh"], df["Saving_€"],
         color="tab:orange", marker="s", label="Saving (€)")
ax1.set_xlabel("Usable capacity (kWh)")
ax1.set_ylabel("Bill reduction (€)", color="tab:orange")
ax1.tick_params(axis="y", labelcolor="tab:orange")

# --- % savings (right y-axis) ---
ax2 = ax1.twinx()
ax2.plot(df["Etot_kWh"], df["Saving_%"],
         color="tab:blue", marker="o", linestyle="--", label="Saving (%)")
ax2.set_ylabel("Bill reduction (%)", color="tab:blue")
ax2.tick_params(axis="y", labelcolor="tab:blue")

# ------------------------------------------------------------
# 4) Annotate the optimal points
# ------------------------------------------------------------
# 95 % of maximum
ax1.scatter(opt95_row["Etot_kWh"], opt95_row["Saving_€"],
            marker="X", s=120, color="black", zorder=3)
ax1.annotate(f"95 % of max:\n{opt95_row.Etot_kWh:.0f} kWh\n€{opt95_row['Saving_€']:.2f}",
             (opt95_row.Etot_kWh, opt95_row["Saving_€"]),
             xytext=(10, -35), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Marginal-benefit threshold
ax1.scatter(opt_margin_row["Etot_kWh"], opt_margin_row["Saving_€"],
            marker="D", s=100, color="green", zorder=3)
ax1.annotate(f"Δ€/ΔkWh < {threshold:.2f}:\n{opt_margin_row.Etot_kWh:.0f} kWh\n€{opt_margin_row['Saving_€']:.2f}",
             (opt_margin_row.Etot_kWh, opt_margin_row["Saving_€"]),
             xytext=(10, 25), textcoords="offset points",
             arrowprops=dict(arrowstyle="->", lw=1))

# Knee point (if available)
if opt_knee_row is not None:
    ax1.scatter(opt_knee_row["Etot_kWh"], opt_knee_row["Saving_€"],
                marker="*", s=140, color="red", zorder=3)
    ax1.annotate(f"Knee (Optimal):\n{opt_knee_row.Etot_kWh:.0f} kWh\n€{opt_knee_row['Saving_€']:.2f}",
                 (opt_knee_row.Etot_kWh, opt_knee_row["Saving_€"]),
                 xytext=(-80, 20), textcoords="offset points",
                 arrowprops=dict(arrowstyle="->", lw=1))

# ------------------------------------------------------------
# 5) Cosmetics
# ------------------------------------------------------------
# merge legends from both axes
lines, labels   = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc="best")

plt.title("Saving vs. battery capacity (Perfect)")
plt.tight_layout()
plt.show()


In [ ]:
#!/usr/bin/env python3
# ==============================================================
# Battery-dispatch optimisation (rolling 24 h horizon, 15-min step)
# PATH B – optimiser has perfect knowledge of actual load
# Cover-as-much-as-possible, no export
# ==============================================================

import cvxpy as cp, numpy as np, pandas as pd, matplotlib.pyplot as plt

# ----------------------------------------------------------------
# 1 FILE PATHS
# ----------------------------------------------------------------
ACTUAL_PATH   = r"C:\Users\gkeep\OneDrive\Desktop\Thesis_coding\Building_Load_15min_Kw.csv"
FORECAST_PATH = r"C:\Users\gkeep\Desktop\Results\DNN\TEST\LSTM_14.03_Test_Results\LSTM_final_predictions.csv"

# ----------------------------------------------------------------
# 2 DATA PREPARATION
# ----------------------------------------------------------------
actual_df = (
    pd.read_csv(ACTUAL_PATH, index_col=0, parse_dates=[0])
      .loc["2013"]
      .loc["2013-06-16":"2013-06-29 23:45"]
      .reset_index()
      .rename(columns={"index": "Timestamp", "WHE (kW)": "Actual Load"})
)

forecast_df = (
    pd.read_csv(FORECAST_PATH)
      .assign(Timestamp=lambda d: pd.to_datetime(d["ds"]),
              Forecast_kW=lambda d: d["LSTM_unscaled"])
      .merge(actual_df[["Timestamp", "Actual Load"]], on="Timestamp", how="inner")
      .sort_values("Timestamp")
      .reset_index(drop=True)
)

def get_price(ts):
    hour = ts.hour
    if 0 <= hour < 2: return 0.10
    elif 2 <= hour < 4: return 0.12
    elif 4 <= hour < 6: return 0.14
    elif 6 <= hour < 8: return 0.16
    elif 8 <= hour < 10: return 0.18
    elif 10 <= hour < 12: return 0.20
    elif 12 <= hour < 14: return 0.22
    elif 14 <= hour < 16: return 0.25
    elif 16 <= hour < 18: return 0.30
    elif 18 <= hour < 20: return 0.40
    elif 20 <= hour < 22: return 0.30
    else: return 0.20

forecast_df["Price"] = forecast_df["Timestamp"].apply(get_price)

# ----------------------------------------------------------------
# 3 PARAMETERS
# ----------------------------------------------------------------

Delta = 0.25   # h, kWh, h⁻¹
Etot = 12  # h, kWh, h⁻¹
C_rate = 0.25   # h, kWh, h⁻¹


Pmax               = Etot * C_rate       # 5 kW
initial_SOC        = 0.10
final_soc_min      = 0.10

demand_charge_rate = 0.0
peak_threshold     = 50.0
peak_penalty_rate  = 15.0
slack_penalty_rate = 1000.0
very_big           = 1e4                # €/kWh penalty for unmet load

# ----------------------------------------------------------------
# 4   ONE-DAY OPTIMISER  (forecast load, no export, minimise unserved)
# ----------------------------------------------------------------
def optimise_one_day(load, price, soc_init, p):
    T = len(load)

    # ── decision variables ───────────────────────────────────────
    P_bat  = cp.Variable(T)             # + discharge / – charge
    SOC    = cp.Variable(T)             # state of charge (fraction)
    Grid   = cp.Variable(T,  nonneg=True)   # grid import (kW)
    Unsv   = cp.Variable(T,  nonneg=True)   # unmet load (kW)
    Peak   = cp.Variable()              # daily peak (kW)
    Excess = cp.Variable(T,  nonneg=True)
    slack  = cp.Variable(T,  nonneg=True)

    # ── build constraints one-by-one ─────────────────────────────
    constr = []

    # 1. Battery power limits
    constr += [P_bat >= -p["Pmax"]]     # charging limit
    constr += [P_bat <=  p["Pmax"]]     # discharging limit

    # 2. Initial SOC
    constr += [SOC[0] == soc_init]

    # 3. SOC update for each time step
    for t in range(1, T):
        constr += [
            SOC[t] == SOC[t-1] - p["Delta"] * P_bat[t] / p["Etot"]
        ]

    # 4. SOC bounds
    constr += [SOC >= 0]
    constr += [SOC <= 1]

    # 5. Define unmet load  (forecast load – battery discharge)
    constr += [Unsv == load - P_bat]

    # 6. Power balance with **no export**
    constr += [Grid == Unsv]   # grid supplies exactly what the battery cannot
    constr += [Grid >= 0]      # import only

    # NEW: when charging (P_bat < 0) its magnitude may not exceed load forecast
    constr += [P_bat >= -load]        # because load ≥ 0
    
    # 7. Optional peak-cap with slack
    peak_cap = 0.0 * p["orig_peak"]     # set >0 if you want a soft cap
    constr += [Grid <= peak_cap + slack]

    # 8. Peak variable must be ≥ every grid value
    constr += [Peak >= Grid]

    # 9. Excess above hard threshold (for penalties)
    constr += [Excess >= Grid - p["peak_threshold"]]
    constr += [Excess >= 0]

    # 10. End-of-day minimum SOC
    constr += [SOC[-1] >= p["soc_final_min"]]

    # ── objective function ───────────────────────────────────────
    energy_cost   = p["Delta"] * cp.sum(cp.multiply(price, Grid))
    demand_charge = p["demand_rate"] * Peak
    soft_cap_pen  = p["Delta"] * p["penalty_rate"] * cp.sum(Excess)
    slack_pen     = p["slack_cost"] * cp.sum(slack)
    unserved_pen  = very_big * p["Delta"] * cp.sum(Unsv)

    objective = cp.Minimize(
        energy_cost + demand_charge + soft_cap_pen + slack_pen + unserved_pen
    )

    # ── solve ────────────────────────────────────────────────────
    prob = cp.Problem(objective, constr)
    # ----- try ECOS, fall back to SCS --------------------------------
    try:
        prob.solve(solver=cp.ECOS, verbose=False,
                max_iters=10_000, abstol=1e-8, reltol=1e-8)
    except cp.error.SolverError:
        prob.solve(solver=cp.SCS, verbose=False,
                max_iters=50_000, eps=1e-5)



    return P_bat.value, SOC.value, Grid.value, SOC.value[-1]


# ----------------------------------------------------------------
# 5 ROLLING-HORIZON LOOP
# ----------------------------------------------------------------
window = pd.date_range("2013-06-16", "2013-06-29 23:45", freq="D")
chunks, soc0 = [], initial_SOC

for d0 in window[:-1]:
    d1   = d0 + pd.Timedelta(days=1) - pd.Timedelta(minutes=15)
    mask = (forecast_df["Timestamp"] >= d0) & (forecast_df["Timestamp"] <= d1)

    Pb, SOCd, Gr, soc0 = optimise_one_day(
        forecast_df.loc[mask, "Actual Load"].values,      # perfect foresight
        forecast_df.loc[mask, "Price"].values,
        soc0,
        dict(Delta=Delta, Etot=Etot, Pmax=Pmax,
            demand_rate=demand_charge_rate,
            peak_threshold=peak_threshold,
            penalty_rate=peak_penalty_rate,
            soc_final_min=final_soc_min,
            orig_peak=forecast_df["Actual Load"].max(),
            slack_cost=slack_penalty_rate)
    )

    blk = forecast_df.loc[mask].copy()
    blk["Battery_kW_perfect"]   = Pb
    blk["SOC"]                  = SOCd
    blk["Grid_Load_kW_perfect"] = Gr
    chunks.append(blk)

df_perf = pd.concat(chunks, ignore_index=True)

# ----------------------------------------------------------------
# 6 COST CALCULATION  (grid already matches actual load slice)
# ----------------------------------------------------------------
price  = df_perf["Price"].values
load   = df_perf["Actual Load"].values
grid   = df_perf["Grid_Load_kW_perfect"].values

cost_no_batt   = Delta * np.sum(load * price)
cost_with_batt = Delta * np.sum(grid * price)
save_eur       = cost_no_batt-cost_with_batt
save_pct       = 100*(cost_no_batt-cost_with_batt)/cost_no_batt
print("────────────────────────────────────────────────")
print(f"{Etot:2d} kWh → €{save_eur:6.2f}  ({save_pct:5.2f} %)")
print("────────────────────────────────────────────────")

# ----------------------------------------------------------------
# 7  PLOTS   (now 5 rows)
# ----------------------------------------------------------------
plt.figure(figsize=(14, 10))          # a bit taller
plt.rcParams.update({
    "font.size":       10,   # base size for all text
    "axes.titlesize":  10,   # figure title
    "axes.labelsize":  12,   # x- and y-axis labels
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.dpi":      120   # crisper output so large fonts stay sharp
})
# 1 ▶ Forecast fed to optimiser
plt.subplot(4,1,1)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(df_perf["Timestamp"], df_perf["Actual Load"],
         label="Perfect Forecast", color="red")
plt.title("Forecast (input to optimiser)"); plt.ylabel("kW"); plt.grid(True); plt.legend()

# 2 ▶ Battery dispatch
plt.subplot(4,1,2)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(df_perf["Timestamp"], df_perf["Battery_kW_perfect"],
         color="orange")
plt.title("Battery Dispatch (+ discharge / – charge)"); plt.ylabel("kW"); plt.grid(True)

# 3 ▶ State-of-Charge
plt.subplot(4,1,3)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(df_perf["Timestamp"], df_perf["SOC"], color="green")
plt.title("State of Charge"); plt.ylabel("fraction"); plt.ylim(0,1); plt.grid(True)

# 4 ▶ Grid import after battery operation
plt.subplot(4,1,4)
ax = plt.gca()
for spine in ['top', 'right', 'left']:
    ax.spines[spine].set_visible(False)
plt.plot(df_perf["Timestamp"], df_perf["Grid_Load_kW_perfect"],
         label="Grid Import (Perfect)", color="blue")
plt.title("Grid Import After Battery"); plt.ylabel("kW"); plt.grid(True); plt.legend()

plt.tight_layout()
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization\Dispatch_plots"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "Perfect.png")
plt.savefig(save_path, dpi=600)
plt.show()


In [ ]:
output_dir = r"C:\Users\gkeep\Desktop\Results\Optimization"
os.makedirs(output_dir, exist_ok=True)
save_path = os.path.join(output_dir, "Perfect_Optimal_Capacity.png")
plt.savefig(save_path, dpi=600)